In [11]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.utils as pyg_utils
import networkx as nx
import matplotlib.pyplot as plt


In [17]:
# dataset name 
dataset_name = 'Cora'

# Load the dataset
dataset = Planetoid(root='./data', name=dataset_name)

# Access the dataset attributes
data = dataset[0]

In [13]:
# Dataset information
print(f'Dataset: {dataset_name}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of node features: {data.num_node_features}')
print(f'Number of node classes: {dataset.num_classes}')

Dataset: Cora
Number of nodes: 2708
Number of edges: 10556
Number of node features: 1433
Number of node classes: 7


In [22]:
# Convert PyTorch Geometric data to NetworkX graph
graph = pyg_utils.to_networkx(data, to_undirected=True)

In [16]:
############ Visualization ####################
# Draws the graph

# plt.figure(figsize=(8, 8))
# nx.draw(graph, node_size=10, node_color='b', edge_color='gray', with_labels=False)
# plt.title('CORA Dataset Graph')
# plt.show()

In [30]:
# print(dataset.data.x)